In [3]:
#完整版程式碼
import requests
import pygsheets

weather_url = 'API_Token'
line_token = "Line_Token"

#建立一個呼叫資料的function，並將資料存入字典裡。
def callapi(url):
    res = requests.get(url)
    data = res.json()
    location = data['records']['locations'][0]['location'][12]
    high_temp = location['weatherElement'][12]
    low_temp = location['weatherElement'][8]
    rain = location['weatherElement'][0]
    uvi = location['weatherElement'][9]
    return {
            'city_name' : location['locationName'],
            'time' : high_temp['time'][0]['startTime'],
            'time_until' : high_temp['time'][0]['endTime'],
            'high_temp' : high_temp['time'][0]['elementValue'][0]['value'],
            'low_temp' : low_temp['time'][0]['elementValue'][0]['value'],
            'rain_' : rain['time'][0]['elementValue'][0]['value'],
            'uvi_' : uvi['time'][0]['elementValue'][0]['value']
    }
#建立一個用line notify通知的function。
def line_notify(msg, line_token):
    line_url = "https://notify-api.line.me/api/notify"
    line_token = line_token
    headers = {
        "Authorization": "Bearer " + line_token, 
        "Content-Type" : "application/x-www-form-urlencoded"
    }
    payload = {'message': msg }
    r = requests.post(line_url, headers = headers, params = payload)
    return


def weather_notify():
    gc = pygsheets.authorize(service_file = r"json_token_file_path")
    wb = gc.open_by_url("google_sheet_url")
    #先呼叫callapi並將天氣資訊存入變數中。
    data = callapi(weather_url)
    time = data['time']
    time_until = data['time_until']
    city_name = data['city_name']
    high_temp = data['high_temp']
    low_temp = data['low_temp']
    rain = data['rain_']
    uvi = data['uvi_']
    
    #根據溫度提供穿著建議。
    if int(low_temp) > 25:
        say = "溫度偏高，適合穿短袖！"
    elif 20 < int(low_temp) < 25:
        say = "溫度適中，記得加件薄外套！"
    elif 15 < int(low_temp) < 20:
        say = "溫度偏低，適合穿長袖和外套！"
    else:
        say = "溫度很低，請注意保暖！"

    #將天氣資訊寫入google sheets中，並用line notify通知。
    wks = wb.worksheet_by_title(city_name)
    date = time[:10]
    hour = time[11:16]
    date_until = time_until[:10]
    hour_until = time_until[11:16]
    col = wks.get_col(1, include_tailing_empty=False)
    last_row = len(col)
    return_lst = [date, hour, date_until, hour_until, high_temp, low_temp, rain, uvi]
    wks.insert_rows(last_row, values=return_lst)

    msg = f" \n {city_name} \n {time}-{time_until}的天氣預報 \n 溫度是 {low_temp}-{high_temp}度C \n 降雨機率是 {rain}% \n UVI是 {uvi} \n {say}"
    line_notify(msg, line_token)

    return print("完成")

weather_notify()

完成
